In [23]:
#!/usr/bin/env python
# coding: utf-8

# In[6]:

import requests   
from bs4 import BeautifulSoup

#[CODE 1]
#네이버 금융 국내증시 상위100의 기업 이름과 url 크롤링
def get_top100(top100_url, top100_name):
    url = 'https://finance.naver.com/sise/sise_quant.nhn'
    result = requests.get(url) # 크롤링
    html = BeautifulSoup(result.content, 'html.parser') # html document 화

    top100 = html.find_all('a', {'class':'tltle'}) # class가 title인 a태그들을 리스트화
    # <a href="/item/main.naver?code=252670" class="tltle">KODEX 200선물인버스2X</a> 등등..

    for i in range(100): # 0부터 까지 반복 총 100번
        url = 'https://finance.naver.com' + top100[i]['href']    
        # https://finance.naver.com/ + item/main.naver?code=252670 등등..
        top100_url.append(url)
    
        company_name = top100[i].string # text와 뭐가 다른건지?..
        # company_name = top100[i].text.strip() 
        # .string 해당 태그 안에 텍스트 노드가 딱 하나만 있을 때 → 그 텍스트를 반환
        # 만약 하위 태그가 섞여 있으면 None 반환할 수 있음
        # text를 쓰는 것이 결과적으로 더 안정적

        top100_name.append(company_name) # top100_name 리스트에 company_name 추가

    return (top100_url, top100_name) # 100개의 url과 name이 들어있는 리스트 2개 반환
    # 근데 어차피 주소값으로 읽어오고 쓸텐데 굳이 리턴할 필요가 있는가?

#[CODE 2]
# 검색할 기업 입력받기
def get_company(top100_name):
    company_name = input('주가를 검색할 기업이름을 입력하세요: ')

    for i in range(101): # 0 부터 100까지 반복 총 101번
        if i == 100: # 일치하는 종목이 없으면 
                     # 0부터 99까지 100개를 돌아서도 없어서 101까지 왔으므로 일치 종목이 없음
            return(100,company_name)
        if company_name == top100_name[i]: # 입력한 이름이 top100_name[i]의 value로 있다면
            return(i,company_name) # index값 리턴

# company_url에 대한 기업 주식 정보 페이지 크롤링
def get_company_stockPage(company_url):    
    result = requests.get(company_url)
    company_stockPage = BeautifulSoup(result.content, "html.parser")
    return company_stockPage

# 기업의 현재 주가 데이터 크롤링
def get_price(company_url):
    company_stockPage = get_company_stockPage(company_url) #[CODE 3]
    no_today = company_stockPage.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"})
    now_price = blind.text
    return now_price


#[CODE 0]
def main():
    top100_url=[]
    top100_name = []

    top100_url, top100_name = get_top100(top100_url, top100_name) #[CODE 1]
    # 100개의 url과 name이 들어있는 list를 받아옴

    print("< 현재 네이버 금융 국내 코스피 거래 상위 100 기업 목록 >")
    print(top100_name) # 이름 출력
    print('')

    i,company = get_company(top100_name)  #[CODE 2]
    # 이름을 보내서 100위 안에 있는지 확인, 근데 굳이 int로 받아야 했을까?
    # String등으로 보내서 입력한 기업이름을 확인할 수 있게 해도 되지 않았을까? 오자등을 확인하도록

    if i == 100:
        print(f"입력한 기업 \'{company}\'은 거래 상위 100 목록에 없습니다. 다시한번 확인해주세요.")
    else :
        now_price = get_price(top100_url[i]) #[CODE 4]
        print('%s 기업의 거래 순위는 %d위로, 현재 주가는 %s원 입니다' % (top100_name[i],i+1, now_price))


if __name__ == '__main__':
     main()


# %%


< 현재 네이버 금융 국내 코스피 거래 상위 100 기업 목록 >
['KODEX 200선물인버스2X', 'KODEX 2차전지산업레버리지', 'KODEX 코스닥150선물인버스', 'KODEX 코스닥150레버리지', 'HJ중공업', '엔케이', 'KODEX 레버리지', 'KODEX 인버스', '영흥', '세진중공업', '삼성 인버스 2X WTI원유 선물 ETN', '삼성전자', 'SOL 조선TOP3플러스', 'TIGER 2차전지TOP10레버리지', '삼성중공업', '한화엔진', '화인베스틸', 'KODEX 2차전지산업', '삼성 인버스 2X 코스닥150 선물 ETN', 'KODEX 코스닥150', '두산에너빌리티', 'TIGER 2차전지소재Fn', 'KODEX 200', 'TIGER 미국S&P500', 'STX엔진', 'KODEX 미국S&P500', 'TIGER 200선물인버스2X', 'TIGER 차이나전기차SOLACTIVE', '한화오션', 'TIGER 조선TOP10', '파미셀', '대우건설', '미래에셋 인버스 2X 코스피200 선물 ETN', 'KODEX 증권', '삼성 레버리지 천연가스 선물 ETN C', 'TIGER 코리아원자력', 'TIGER 화장품', '미래에셋증권', '한국전력', 'N2 블룸버그 2X 천연가스 선물 ETN(H)', 'ACE 미국30년국채액티브(H)', 'TIGER 2차전지TOP10', '모나미', 'TIGER 200 중공업', 'TIGER 미국테크TOP10 INDXX', 'KODEX 2차전지핵심소재10', 'TP', '이수페타시스', 'TIGER 차이나항셍테크', 'KODEX 미국AI전력핵심인프라', 'SK오션플랜트', '삼성 인버스 2X 코스피200 선물 ETN', 'KODEX 200타겟위클리커버드콜', 'KODEX AI전력핵심설비', '두산퓨얼셀', '에이프로젠', '상상인증권', '웅진', 'ACE 미국빅테크TOP7 Plus', 'KODEX 차이나휴머노이드로봇', '한화손해보험', '삼성 레버리지 WTI원유 선물 ETN', 